In [1]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg

from math import pi

from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

In [2]:
rospy.init_node('bf_ur5', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'gripper']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])
planning_frame = move_group.get_planning_frame()

In [4]:
eef_link = move_group.get_end_effector_link()

In [5]:
robot_state = robot.get_current_state()
robot_state

joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, robotiq_85_left_inner_knuckle_joint, robotiq_85_left_finger_tip_joint,
  robotiq_85_left_knuckle_joint, robotiq_85_left_finger_joint, robotiq_85_right_inner_knuckle_joint,
  robotiq_85_right_finger_tip_joint, robotiq_85_right_knuckle_joint, robotiq_85_right_finger_joint]
  position: [-0.0028462404650468898, 0.01584641353191696, 0.0001053063594111947, 0.005020391138707225, -0.0006049482404621997, -0.018570347900235262, -7.830130440034111e-06, 7.830130440034111e-06, -7.830130440034111e-06, 0.0, -7.830130440034111e-06, 7.830130440034111e-06, -7.830130440034111e-06, 0.0]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
atta

In [6]:

joint_goal = move_group.get_current_joint_values()
joint_goal

[-0.002846240465047778,
 0.015846413531917847,
 0.0001053063594111947,
 0.005020391138706337,
 -0.0006049482404621997,
 -0.018570347900235262]

In [7]:
def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [8]:
get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 91
    nsecs: 252000000
  frame_id: "world"
pose: 
  position: 
    x: 0.815670784314
    y: 0.196629187965
    z: 0.981540781535
  orientation: 
    x: -0.707897817191
    y: -0.706313841751
    z: 0.000842771433638
    w: 0.000852703814737


In [9]:
import geometry_msgs.msg
from geometry_msgs.msg import Quaternion, Point

def bo_move_position( position, orientation ):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.orientation = orientation
    pose_goal.position = position
    move_group.set_pose_target(pose_goal)
    plan = move_group.go(wait=True)
    move_group.stop()
    move_group.clear_pose_targets()


In [20]:
orientation_0 = Quaternion(-0.707732860392,
                          -0.706466425662,
                           0.00311631615142,
                           0.00311067637957)

quat = quaternion_from_euler(0, 0, pi/2.0)
ori_0 = Quaternion( quat[0], quat[1], quat[2], quat[3])

quat = quaternion_from_euler(0, 0, -pi/2.0)
ori_1 = Quaternion( quat[0], quat[1], quat[2], quat[3])

Y_ENTRY_POINT = 0.44
Y_INSERT_LENGTH = 0.15


In [21]:
# rack coordinate


rack_0U = [ Point(-0.44, Y_ENTRY_POINT, 0.998+0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998+0.376)
          ]

rack_0L = [ Point(-0.44, Y_ENTRY_POINT, 0.998-0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998-0.376)
          ]

rack_1U = [ Point(-0.44, -Y_ENTRY_POINT, 0.998+0.376),
           Point(-0.22, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.0, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.22, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.44, -Y_ENTRY_POINT, 0.998+0.376)
          ]

rack_1L = [ Point(-0.44, -Y_ENTRY_POINT, 0.998-0.376),
           Point(-0.22, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.0, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.22, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.44, -Y_ENTRY_POINT, 0.998-0.376)
          ]


In [24]:
# go to the initial position 

for point in rack_0U:
    bo_move_position(point, ori_0)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z), ori_0)
    bo_move_position(point, ori_0 )

for point in reversed(rack_0L):
    bo_move_position(point, ori_0)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z), ori_0)
    bo_move_position(point, ori_0)

# go to the initial position
bo_move_position(rack_0U[0], ori_0)


In [ ]:
# when we go to the opposite side of the rack, orientation should be reconsidered.
    
for point in rack_1U:
    bo_move_position(point, ori_1)
    bo_move_position(Point(point.x, point.y-Y_INSERT_LENGTH, point.z), ori_1)
    bo_move_position(point, ori_1)

for point in reversed(rack_1L):
    bo_move_position(point, ori_1)
    bo_move_position(Point(point.x, point.y-Y_INSERT_LENGTH, point.z), ori_1)
    bo_move_position(point, ori_1)

# go to the initial position
bo_move_position(rack_0U[0])

Constraint setup is very tricky in moveit. Instrcutions are not well-documented.

In [ ]:
# end effector tilt tolerance setup

constraints = Constraints()

constraints.name = "tilt_constraint"

tilt_constraint = OrientationConstraint()

tilt_constraint.header.frame_id = "base_link"

tilt_constraint.link_name = "gripper"

tilt_constraint.absolute_x_axis_tolerance = 0.45
tilt_constraint.absolute_y_axis_tolerance = 0.45
tilt_constraint.absolute_z_axis_tolerance = 3.6
tilt_constraint.weight = 1
constraints.orientation_constraints.append(tilt_constraint)
move_group.set_path_constraints(constraints)

for cartesian movement sample code :  https://github.com/byeongkyu/ros_tutorials3/blob/master/mobile_manipulator_robot_moveit_client/src/d4_move_cartesian_path.py